# Set Up + Find Condition Codes

In [ ]:
# Import packages and codes
%load_ext google.cloud.bigquery
from datetime import date
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil
from IPython.display import display, HTML
import os
import subprocess

In [ ]:
dataset = os.getenv("WORKSPACE_CDR")
dataset

In [ ]:
# OMOP Codes for Medications

# Metformin = 1503297
medications = ['1503297']

# Build Dataset

In [ ]:
dataset_66101830_drug_sql = """
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_exposure.verbatim_end_date,
        d_exposure.drug_type_concept_id,
        d_type.concept_name as drug_type_concept_name,
        d_exposure.stop_reason,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.days_supply,
        d_exposure.sig,
        d_exposure.route_concept_id,
        d_route.concept_name as route_concept_name,
        d_exposure.lot_number,
        d_exposure.visit_occurrence_id,
        d_visit.concept_name as visit_occurrence_concept_name,
        d_exposure.drug_source_value,
        d_exposure.drug_source_concept_id,
        d_source_concept.concept_name as source_concept_name,
        d_source_concept.concept_code as source_concept_code,
        d_source_concept.vocabulary_id as source_vocabulary,
        d_exposure.route_source_value,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN  (
                    SELECT
                        DISTINCT ca.descendant_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                    JOIN
                        (
                            select
                                distinct c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (
                                    select
                                        cast(cr.id as string) as id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                    WHERE
                                        concept_id IN ("""+",".join([str(cid) for cid in medications])+""")
                                        AND full_text LIKE '%_rank1]%'
                                ) a 
                                    ON (
                                        c.path LIKE CONCAT('%.',
                                    a.id,
                                    '.%') 
                                    OR c.path LIKE CONCAT('%.',
                                    a.id) 
                                    OR c.path LIKE CONCAT(a.id,
                                    '.%') 
                                    OR c.path = a.id) 
                                WHERE
                                    is_standard = 1 
                                    AND is_selectable = 1
                                ) b 
                                    ON (
                                        ca.ancestor_id = b.concept_id
                                    )
                            )
                        )  
                        AND (
                            d_exposure.PERSON_ID IN (
                                SELECT
                                    distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT ca.descendant_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                                                    JOIN
                                                        (
                                                            select
                                                                distinct c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN ("""+",".join([str(cid) for cid in medications])+""") 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) b 
                                                                    ON (
                                                                        ca.ancestor_id = b.concept_id
                                                                    )
                                                            ) 
                                                            AND is_standard = 1
                                                        )
                                                ) criteria 
                                            ) 
                                        ))) d_exposure 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_standard_concept 
                                    ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_type 
                                    ON d_exposure.drug_type_concept_id = d_type.concept_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_route 
                                    ON d_exposure.route_concept_id = d_route.concept_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                    ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_visit 
                                    ON v.visit_concept_id = d_visit.concept_id 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_source_concept 
                                    ON d_exposure.drug_source_concept_id = d_source_concept.concept_id"""

dataset_66101830_drug_df = pd.read_gbq(
    dataset_66101830_drug_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

In [ ]:
dataset_66101830_drug_df

In [ ]:
# See how many duplicates there are
dataset_66101830_drug_df[dataset_66101830_drug_df.duplicated()]

In [ ]:
# Drop the duplicates
metformin_med = dataset_66101830_drug_df.drop_duplicates()
metformin_med

In [ ]:
# See how many unique patients are in the dataset
metformin_med['person_id'].nunique()

In [ ]:
# Save dataset as a csv file

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = metformin_med   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'Metformin_Medication_AG_10122023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

In [ ]:
# Count the frequency of each drug in the dataset
dataset_66101830_drug_df['standard_concept_name'].sort_values(ascending=True).value_counts().head(200)

# Upload the whole dataset as parquet file to Google Bucket

In [ ]:
import pyarrow.parquet as pq

In [ ]:
pip install fastparquet

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = dataset_66101830_drug_df   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'Metformin_Cases_AG_10122023.parquet.gzip'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a parquet file in the same workspace as the notebook
my_dataframe.to_parquet(destination_filename, compression='gzip')

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy parquet file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

# Data Cleaning

In [ ]:
# Upload metformin dataframe
# This snippet assumes you run setup first

# This code copies file in your Google Bucket and loads it into a dataframe

# Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
name_of_file_in_bucket = 'Metformin_Medication_AG_10122023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
my_dataframe = pd.read_csv(name_of_file_in_bucket)
my_dataframe

In [ ]:
# Find number of unique PIDs
my_dataframe['person_id'].nunique()

In [ ]:
#Sort unique person ids by their earliset condition date.
def sort_unique_by_min_date(df): 
    min_dates_diags = df.sort_values(["person_id","drug_exposure_start_datetime"]).groupby("person_id", as_index=False).first()
    return min_dates_diags

In [ ]:
min_dates_diags = sort_unique_by_min_date(my_dataframe)
min_dates_diags

In [ ]:
# Save as csv file (min date)
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = min_dates_diags   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'Metformin_Medication_Min_Date_AG_10122023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

In [ ]:
# This snippet assumes that you run setup first

# This code lists objects in your Google Bucket

# Get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))

In [ ]:
my_dataframe['person_id'].duplicated(keep='first')

In [ ]:
my_df = my_dataframe.drop_duplicates('person_id')

In [ ]:
my_df = my_df.reset_index(drop=True)
my_df

In [ ]:
my_df['person_id'].nunique()

In [ ]:
person_list = my_df['person_id']

In [ ]:
# Upload demo data
# This snippet assumes you run setup first

# This code copies file in your Google Bucket and loads it into a dataframe

# Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
name_of_file_in_bucket = 'demographic_all.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
demo = pd.read_csv(name_of_file_in_bucket)
demo.head()

In [ ]:
demo = demo[demo['person_id'].isin(person_list)]
demo

In [ ]:
demo = demo.reset_index()

cleaned_demo = demo[['person_id','year_of_birth','gender_source_value','sex_at_birth_source_value','race_source_value', 'ethnicity_source_value']]


cleaned_demo = cleaned_demo.rename(columns={'person_id':"Count",
                                            'year_of_birth':"Age",
                                            'gender_source_value': "Gender",
                                            'sex_at_birth_source_value':"Sex at Birth",
                                            'race_source_value':"Race",
                                            'ethnicity_source_value':"Hispanic"})


for row in (range(cleaned_demo.shape[0])):
    for col in (range(cleaned_demo.shape[1])):
        if cleaned_demo.iloc[row,col] == "PMI_Skip":
            cleaned_demo.iloc[row,col] = "Skip"
        if cleaned_demo.iloc[row,col] in ["PMI_PreferNotToAnswer",
                                          "SexAtBirth_Intersex",
                                          "SexAtBirth_SexAtBirthNoneOfThese",
                                          "No matching concept",
                                          "GenderIdentity_AdditionalOptions",
                                          "GenderIdentity_GeneralizedDiffGender",
                                          "GenderIdentity_NonBinary",
                                          "GenderIdentity_Transgender"]:
            cleaned_demo.iloc[row,col] = "Unspecified"


for x in range(len(cleaned_demo)):
    birth_year = cleaned_demo.at[x,'Age']
    cleaned_demo.at[x,'Age'] = date.today().year - birth_year

bins = [0,29,49,69,89,1000]
labels = ['0-29','30-49','50-69','70-89','90+']
cleaned_demo['Age Group'] = pd.cut(cleaned_demo['Age'], bins=bins, labels=labels, right=False)

# Data Summarization and Visualization

## Sex at Birth & Gender Identity

In [ ]:
count_gender_gr = cleaned_demo[['Count','Gender']].groupby(['Gender'], as_index=False).count()
count_gender_gr['%'] = 100 * count_gender_gr['Count'] / len(cleaned_demo)
display(count_gender_gr)

fig1, ax1 = plt.subplots()
ax1.pie(cleaned_demo['Gender'].value_counts(), labels=['GenderIdentity_Woman','GenderIdentity_Man','Unspecified','Skip'], 
        autopct='%1.1f%%', startangle=90, pctdistance=0.5)
ax1.axis('equal')
fig = plt.gcf().gca().add_artist(plt.Circle((0,0),0.70,fc='white'))
plt.tight_layout()
#rcParams['figure.figsize'] = (10,10)
plt.show()

## Race and Ancestry

In [ ]:
count_race_gr = cleaned_demo[['Count','Race']].groupby(['Race'], as_index=False).count()
count_race_gr['%'] = 100 * count_race_gr['Count'] / len(cleaned_demo)
display(count_race_gr)

sns.barplot(x='Count', y='Race',data=cleaned_demo[['Count','Race']].groupby(['Race'], as_index=False).count());

## Ethnicity

In [ ]:
count_eth_gr = cleaned_demo[['Count','Hispanic']].groupby(['Hispanic'], as_index=False).count()
count_eth_gr['%'] = 100 * count_eth_gr['Count'] / len(cleaned_demo)
display(count_eth_gr)

fig1, ax1 = plt.subplots()
ax1.pie(cleaned_demo['Hispanic'].value_counts(),  #labels=['Not HLS','Hispanic, Latino, or Spanish', 'Skip', 'None of These', 'Prefer not to Answer'],
        autopct='%1.1f%%', startangle=0, pctdistance=0.5)
ax1.axis('equal')
fig = plt.gcf().gca().add_artist(plt.Circle((0,0),0.70,fc='white'))
plt.tight_layout()
#rcParams['figure.figsize'] = (10,10)
plt.show();

## Age Distribution

In [ ]:
count_age_gr = cleaned_demo[['Count','Age Group']].groupby(['Age Group'], as_index=False).count()
count_age_gr['%'] = 100 * count_age_gr['Count'] / len(cleaned_demo)
display(count_age_gr)


#rcParams['figure.figsize'] = (20,10)
sns.histplot(cleaned_demo['Age']);

summary = cleaned_demo.groupby('Age Group')['Age'].describe()[['min', '25%', '50%', '75%', 'max']]
summary

In [ ]:
tab = cleaned_demo.groupby(['Age']).size()
pd.set_option('display.max_rows', None)
print(tab)

In [ ]:
sns.histplot(cleaned_demo['Age'], bins=30)